We start by adding the necessary folders to the current working path.

In [ ]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add 'PRA3D' folder to current working path in order to access the functions
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/PRA3D")

# Overview

In the following notebook we compute the Polar Rotation Angle ($ \mathrm{PRA} $) on the three-dimensional, steady ABC-flow. We emphasize that in 3D, the $ \mathrm{PRA} $  does not objectively compute elliptic flow features, unless the underlying vector field is inherently objective. The velocity field is analytically defined. The notebook is structured as follows:

1. Define analytic velocity-field of steady ABC-flow.
<br />
2. Define computational parameters.
 <br />
3. Define spatio-temporal domain.
 <br />
5. $ \mathrm{PRA} $ :

    * Compute gradient of flow map $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $. The gradient of the flow map is computed by solving the system of coupled ODEs:
    \begin{align*}
    \dfrac{d}{dt}\mathbf{F}_{t_0}^{t}(\mathbf{x}_0) &= \mathbf{v}(\mathbf{F}_{t_0}^{t}(\mathbf{x}_0),t) \\
    \dfrac{d}{dt}\mathbf{\nabla F}_{t_0}^{t}(\mathbf{x}_0) &= \mathbf{\nabla v}(\mathbf{x},t)\mathbf{\nabla F}_{t_0}^{t}(\mathbf{x}_0)
    \end{align*}
    <br />
    
    * Compute PRA from left and right singular vectors $ \xi_i, \eta_i $ (with i = 1, 2, 3)  of $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $ according to:

    \begin{equation}
        \mathrm{PRA}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{2} (\sum_{i=1}^{3} \langle \xi_i(\mathbf{x}_0;t_0, t_N), \eta_i(\mathbf{x}_0;t_0, t_N) \rangle - 1)
    \end{equation}

# Define analytic velocity field

Here we define the analytic steady velocity of the ABC-flow.

\begin{equation}
\begin{pmatrix} \dot{x}(t) \\ \dot{y}(t) \\ \dot{z}(t) \end{pmatrix} = \begin{pmatrix} \sqrt{3}\sin(z)+\cos(y) \\ \sqrt{2}\sin(x)+\sqrt{3}\cos(z) \\ \sin(y)+\sqrt{2}\cos(x) \end{pmatrix}
\end{equation}.

In [ ]:
def velocity(x, y, z):

    u = sqrt(3)*np.sin(z)+np.cos(y)
    v = sqrt(2)*np.sin(x)+sqrt(3)*np.cos(z)
    w = np.sin(y)+sqrt(2)*np.cos(x)
    
    return np.array([u, v, w])

def grad_velocity(x, y, z):
    
    grad_vel = np.zeros((x.shape[0], 3, 3))

    dudx = 0
    dudy = -np.sin(y)
    dudz = sqrt(3)*np.cos(z)
    
    dvdx = sqrt(2)*np.cos(x)
    dvdy = 0
    dvdz = -sqrt(3)*np.sin(z)
    
    dwdx = -sqrt(2)*np.sin(x)
    dwdy = np.cos(y)
    dwdz = 0
    
    grad_vel[:,0,0] = dudx
    grad_vel[:,1,0] = dvdx
    grad_vel[:,2,0] = dwdx
    
    grad_vel[:,0,1] = dudy
    grad_vel[:,1,1] = dvdy
    grad_vel[:,2,1] = dwdy
    
    grad_vel[:,0,2] = dudz
    grad_vel[:,1,2] = dvdz
    grad_vel[:,2,2] = dwdz
    
    return grad_vel

# Computational parameters and analytic velocity field

Here we define the computational parameters.

In [ ]:
# import numpy
import numpy as np

# import math tools
from math import sqrt, pi

# number of cores to be used for parallel computing
Ncores = 7

# periodic boundary conditions
periodic_x = True
periodic_y = True
periodic_z = True
periodic = [periodic_x, periodic_y, periodic_z]

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
# Initial time (in days)
t0 = 0 # float

# Final time (in days)
tN = 50 # float

# Time step-size (in days)
dt = 0.1 # float

# NOTE: For computing the backward FTLE field tN < t0 and dt < 0.

time = np.arange(t0, tN+dt, dt) # shape (Nt,)

# Length of time interval (in days)
lenT = abs(tN-t0) # float

# boundaries
xmin = 0 # float
xmax = 2*np.pi # float
ymin = 0 # float
ymax = 2*np.pi # float
zmin = 0 # float
zmax = 2*np.pi # float

# Resolution of meshgrid
Ny = 200 # int
Nx = 200 # int
Nz = 200 # int

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True) # array (Nx, )
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True) # array (Ny, )
z_domain = np.linspace(zmin, zmax, Nz, endpoint = True) # array (Nz, )

dx = x_domain[1]-x_domain[0] # float
dy = y_domain[1]-y_domain[0] # float
dz = z_domain[1]-z_domain[0] # float

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, z_domain) # array (Ny, Nx, Nz)

Ny = X_domain.shape[0] # int
Nx = X_domain.shape[1] # int
Nz = X_domain.shape[2] # int

# Define Integration

In [ ]:
# import package for progress bar
from tqdm.notebook import tqdm

def integration_dDF_dt(time, x):
    
    # reshape x
    x = x.reshape(3, -1) # reshape array (3, Nx*Ny*Nz)
    
    # Initialize arrays for flow map and derivative of flow map
    Fmap = np.zeros((len(time), 3, x.shape[1])) # array (Nt, 3, Nx*Ny*Nz)
    dFdt = np.zeros((len(time)-1, 3, x.shape[1])) # array (Nt-1, 3, Nx*Ny*Nz)
    DF = np.zeros((len(time), 3, 3, x.shape[1])) # array (Nt, 3, Nx*Ny*Nz)
    
    # Step-size
    dt = time[1]-time[0] # float
    
    counter = 0 # int

    # initial conditions
    Fmap[counter,:,:] = x
    DF[counter,0,0,:] = 1
    DF[counter,1,1,:] = 1
    DF[counter,2,2,:] = 1
    
    # Runge Kutta 4th order integration with fixed step size dt
    for t in tqdm(time[:-1]):
        
        if t%1 == 0:
            print('Percentage completed: ', np.around(t/time[-1], 3))
        
        Fmap[counter+1,:, :], dFdt[counter,:,:], DF[counter+1,:,:,:] = RK4_step(t, Fmap[counter,:, :], DF[counter, :,:,:], dt)
    
        counter += 1
    
    return Fmap, dFdt, DF

def RK4_step(t, x1, DF1, dt):
    
    t0 = t # float
    
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = velocity(x1[0,:], x1[1,:], x1[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute gradient of velocity
    grad_k1 = np.zeros(DF1.shape) # array(3,3, Nx*Ny*Nz)
    
    grad_vel = grad_velocity(x1[0,:], x1[1,:], x1[2,:]) # array(3,3, Nx*Ny*Nz)
    
    for i in range(grad_k1.shape[2]):
        grad_k1[:,:,i] = dt*np.dot(grad_vel[i,:,:], DF1[:,:,i])
    
    # compute derivative
    k1 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the first midpoint.
    x2 = x1 + .5 * k1 # array(3, Nx*Ny*Nz)
    
    DF2 = DF1 + .5 * grad_k1 # array(3,3, Nx*Ny*Nz)
     
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the first midpoint.
    x_prime = velocity(x2[0,:], x2[1,:], x2[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute gradient of velocity
    grad_k2 = np.zeros(DF1.shape) # array(3,3, Nx*Ny*Nz)
    
    grad_vel = grad_velocity(x2[0,:], x2[1,:], x2[2,:]) # array(3,3, Nx*Ny*Nz)
    
    for i in range(grad_k2.shape[2]):
        grad_k2[:,:,i] = dt*np.dot(grad_vel[i,:,:], DF2[:,:,i])
    
    # compute derivative
    k2 = dt * x_prime # array(3, Nx*Ny*Nz)

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2 # array(3, Nx*Ny*Nz)
    
    DF3 = DF1 + .5 * grad_k2 # array(3,3, Nx*Ny*Nz)
    
    # Update time
    t = t0 + .5*dt # float
    
    # Compute x_prime at the second midpoint.
    x_prime = velocity(x3[0,:], x3[1,:], x3[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute gradient of velocity
    grad_k3 = np.zeros(DF1.shape) # array(3,3, Nx*Ny*Nz)
    
    grad_vel = grad_velocity(x3[0,:], x3[1,:], x3[2,:]) # array(3,3, Nx*Ny*Nz)
    
    for i in range(grad_k3.shape[2]):
        grad_k3[:,:,i] = dt*np.dot(grad_vel[i,:,:], DF3[:,:,i])
    
    # compute derivative
    k3 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Update position at the endpoint.
    x4 = x1 + k3 # array(3, Nx*Ny*Nz)
    
    DF4 = DF1 + grad_k3 # array(3,3, Nx*Ny*Nz)
    
    # Update time
    t = t0+dt # float
    
    # Compute derivative at the end of the time-step.
    x_prime = velocity(x4[0,:], x4[1,:], x4[2,:]) # array(3, Nx*Ny*Nz)
    
    # compute gradient of velocity
    grad_k4 = np.zeros(DF1.shape) # array(3,3, Nx*Ny*Nz)
    
    grad_vel = grad_velocity(x4[0,:], x4[1,:], x4[2,:]) # array(3,3, Nx*Ny*Nz)
    
    for i in range(grad_k4.shape[2]):
        grad_k4[:,:,i] = dt*np.dot(grad_vel[i,:,:], DF4[:,:,i])
    
    # compute derivative
    k4 = dt * x_prime # array(3, Nx*Ny*Nz)
    
    # Compute RK4 derivative
    y_prime_update = 1.0 / 6.0*(k1 + 2 * k2 + 2 * k3 + k4) # array(3, Nx*Ny*Nz)
    
    dDFdt_update = 1.0 / 6.0*(grad_k1 + 2 * grad_k2 + 2 * grad_k3 + grad_k4) # array(3,3, Nx*Ny*Nz)
    
    # Integration y <-- y + y_primeupdate
    y_update = x1 + y_prime_update # array(3, Nx*Ny*Nz)

    # Integration DF <-- DF + dDFdt_update
    DF_update = DF1 + dDFdt_update # array(3,3, Nx*Ny*Nz)
    
    return y_update, y_prime_update/dt, DF_update

# PRA

The computation of the $ \mathrm{PRA} $ is not done over the 3D meshgrid but only over the 2D faces of the cube $ [0,2\pi]^3 $.
We iterate over all initial conditions on the faces of the cube and calculate the gradient of the flow map. From the left and right singular vectors of the gradient of the flow map we then compute the $ \mathrm{PRA} $.

In [ ]:
# Import function to compute Polar Rotation Angle (PRA)
from ipynb.fs.defs.PRA import _PRA

# Import package for parallel computing
from joblib import Parallel, delayed

def compute_PRA(x0, y0, z0):
    
    X0 = np.array([x0, y0, z0]) # array (3, Nx*Ny*Nz)
    
    DF = integration_dDF_dt(time, X0)[-1] # array (Nt, 3, 3, Nx*Ny*Nz)
    
    PRA = []
    
    for i in range(DF.shape[3]):
        PRA.append(_PRA(DF[-1,:,:,i]))
    
    return PRA

# Split x0,y0,z0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

## XY-plane

Compute $ \mathrm{PRA} $ over the 2D meshgrid on the XY-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_xy = X_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )
y0_xy = Y_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )
z0_xy = Z_domain[:,:,-1].ravel() # array (Nx*Ny*Nz, )

x0_batch = list(split(x0_xy, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_xy, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_xy, Ncores)) # list (Nx*Ny*Nz)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_PRA)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of PRA computation on xy-plane
PRA_xy = results[0]
for res in results[1:]:
    PRA_xy = np.append(PRA_xy, res)

# reshape vectorized arrays to structured array
X0_xy = np.array(x0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Y0_xy = np.array(y0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Z0_xy = np.array(z0_xy).reshape(Ny,Nx) # array (Ny, Nx)
PRA_xy = np.array(PRA_xy).reshape(Ny,Nx) # array (Ny, Nx)

## XZ-plane

Compute $ \mathrm{PRA} $ over the 2D meshgrid on the XZ-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_xz = X_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )
y0_xz = Y_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )
z0_xz = Z_domain[0,:,:].ravel() # array (Nx*Ny*Nz, )

x0_batch = list(split(x0_xz, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_xz, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_xz, Ncores)) # list (Nx*Ny*Nz)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_PRA)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of PRA computation on xz-plane
PRA_xz = results[0]
for res in results[1:]:
    PRA_xz = np.append(PRA_xz, res)

# reshape vectorized arrays to structured array
X0_xz = np.array(x0_xz).reshape(Nz,Nx) # array (Nz, Nx)
Y0_xz = np.array(y0_xz).reshape(Nz,Nx) # array (Nz, Nx)
Z0_xz = np.array(z0_xz).reshape(Nz,Nx) # array (Nz, Nx)
PRA_xz = np.array(PRA_xz).reshape(Nz,Nx) # array (Nz, Nx)

## YZ-plane

Compute $ \mathrm{PRA} $ over the 2D meshgrid on the YZ-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_yz = X_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )
y0_yz = Y_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )
z0_yz = Z_domain[:,0,:].ravel() # array (Nx*Ny*Nz, )

x0_batch = list(split(x0_yz, Ncores)) # list (Nx*Ny*Nz)
y0_batch = list(split(y0_yz, Ncores)) # list (Nx*Ny*Nz)
z0_batch = list(split(z0_yz, Ncores)) # list (Nx*Ny*Nz)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_PRA)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract results of PRA computation on yz-plane
PRA_yz = results[0]
for res in results[1:]:
    PRA_yz = np.append(PRA_yz, res)

# reshape vectorized arrays to structured array
X0_yz = np.array(x0_yz).reshape(Nz,Ny) # array (Nz, Ny)
Y0_yz = np.array(y0_yz).reshape(Nz,Ny) # array (Nz, Ny)
Z0_yz = np.array(z0_yz).reshape(Nz,Ny) # array (Nz, Ny)
PRA_yz = np.array(PRA_yz).reshape(Nz,Ny) # array (Nz, Ny)

In [ ]:
# Import plotly for 3D figures
from plotly import graph_objs as go

# define minimum and maximum values for colorbar
min_value = min(np.min(PRA_yz),min(np.min(PRA_xy), np.min(PRA_xz)))
max_value = max(np.max(PRA_yz),max(np.max(PRA_xy), np.max(PRA_xz)))

# create figure
fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = PRA_xy, showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = PRA_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = PRA_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*pi, surfacecolor = PRA_xy, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*pi, z = Z0_xz, surfacecolor = PRA_xz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
fig.add_trace(go.Surface(x = X0_yz+2*pi, y = Y0_yz, z = Z0_yz, surfacecolor = PRA_yz, showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

# camera
camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
fig.update_layout(scene_camera=camera)

fig.show()

The $ \mathrm{PRA} $ admits tubular level surfaces that closely approximate the invariant tori. Codimension-two level sets of the $ \mathrm{PRA} $  are periodic material curves at the cores of the elliptic regions. As in earlier examples, outside the elliptic islands formed by these closed level surfaces, $ \mathrm{PRA} $  levels exhibit small-scale variations due to sensitive dependence of the rotation angle on the initial conditions.

# References

[1] Farazmand, M., & Haller, G. (2016). Polar rotation angle identifies elliptic islands in unsteady dynamical systems. Physica D: Nonlinear Phenomena, 315, 1-12.